<a href="https://www.kaggle.com/code/khhaledahmaad/sql-to-pandas-v1?scriptVersionId=143573896" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

__Unlocking the Power of SQL Analytics Functions in Pandas: A Data Science Journey!__

<center>
    <img src="https://www.scaler.com/topics/images/sql-vs-pandas-thumbnail.webp" width=500 alt="sql vs pandas" />
</center>

<div style="text-align: center;">
    <txt>Image Source:<txt/>
    <a href="https://www.scaler.com/topics/pandas/sql-vs-pandas/">SCALER Topics</a>
<div/>

### Introduction
In the world of data science, the ability to seamlessly integrate powerful `SQL` analytics functions with the flexibility and versatility of `Pandas` in Python can be a game-changer. In this notebook, we'll embark on a journey to demonstrate how the capabilites of some of the formidable `SQL` analytics tools can be harmoniously transformed to supercharge your data manipulation and analysis capabilities in `Pandas`.

### Objective
The key objective of this demo is to demonstrate how efficient `Pandas` logics/fucntions can be formulated to achieve results like `SQL` style quick queries and results for the following:
1. Joining and Mergeing datasets
2. Advanced Analytics functions
3. Nested and Repeated data

__Note:__ 
1. In terms of querying large datasets in the cloud, it is much more efficient to use `SQL` queries for all sorts of merging and aggregations before transforming the datasets into Pandas dataframes. The main purpose of this notebook is to envision hypothetical scenarios where we need to perform similar SQL-like operations using Pandas when dealing with separate datasets from other sources, such as `CSV` files. `Pandas` does not currently have any built-in functionalities that allow us to use `SQL` queries directly. Additionally, when dealing with SQL-based databases or data warehouses, we can efficiently and quickly retrieve data, merge it as necessary, and even apply advanced aggregations, saving `CPU` usage and memory by utilising SQL at the query level.

2. There are many ways and coding logics to achieve the results shown in this notebook. The examples provided are some of the easier ways to do so with the less possible codes.

### Prerequisite 
1. Basic understanding of `SQL`
2. Basic understanding of `Pandas`
3. Basic understainding of various `data structures` in programming
4. Basic understanding of `SQL` style joins and their types (`INNER`, `OUTER`/`FULL`, `LEFT`, `RIGT`)
5. Basic understanding of `databases`, `cloud`, and `API`

### Background
1. When dealing with data, you may come across datasets that contain columns or attributes sharing identical values. In the context of Relational Database Management Systems, these are referred to as `foreign keys` [1]. For the sake of simplicity, we will simply call them `keys`." These keys exhibit identical values across various datasets, enabling the merging of different datasets into a single dataset based on the matching key values of each data point.

2. Advanced analytics functions are frequently employed in SQL-based database queries. These functions enable the grouping of data and the application of sophisticated operations to various data points within the same groups. Here are some common analytics functions used in SQL [2].

    2.1. __Analytic aggregate functions__

    `MIN()` or `MAX()` - Returns the minimum (or maximum) of input values

    `AVG()` or `SUM()` - Returns the average (or sum) of input values

    `COUNT()` - Returns the number of rows in the input

    2.2. __Analytic navigation functions__

    `FIRST_VALUE()` or `LAST_VALUE()` - Returns the first (or last) value in the input

    `LEAD()` and `LAG()` - Returns the value on a subsequent (or preceding) row

    2.3. __Analytic numbering functions__

    `ROW_NUMBER()` - Returns the order in which rows appear in the input (starting with 1)

    `RANK()` - All rows with the same value in the ordering column receive the same rank value, where the next row receives a rank value which increments by the number of rows with the previous rank value.
    
3. In the most basic terms, `Nested` data refers to data structures resembling `JSON` objects or `Python` dictionaries that exist within the values of a column in a dataset. When there are multiple instances of these `Nested` objects enclosed within an array or list (denoted by `[]`), it is simply referred to as `Repeated` data. `Nested` data can be likened to a row of a table inside another table, whereas `Repeated` data represents multiple rows of a table nested inside another table. These concepts of `Nested` and `Repeated` data are commonly encountered in cloud-based databases or data retrieved from APIs, where the output data is often stored in `JSON` format.

### Setting the Stage: Importing Libraries and Data
In this demo, we've used the following:
1. `Google Cloud Bigquery`, a cloud-based `Big Data` warehouse provided by `Google`.

    __N.B.:__ In this notebook, we used `Kaggle's` public dataset BigQuery integration

2. `SQL` to query the `BigQuery` public datasets. 
3. `Pandas` for data manipulation in `Python`.
4. `JSON` to normalise `JSON` like objects

In [ ]:
# Install the required libraries using the following commands (optional)
# !pip install google-cloud-bigquery
# !pip install pandas

In [ ]:
# Import the libararies
import pandas as pd
import json
from google.cloud import bigquery

In [ ]:
# Create a bigquery client object
client = bigquery.Client()

We can set the `job_config` in the `client` object when passing the query to restrict the memory usage by the query to `1GB`.

In [ ]:
# Create a safe job_config
# only run the query if it's less than 1 GB
ONE_GB = 1000*1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_GB)

### 1. Joining and mergeing datasets
Here, we will merge different datasets using both `SQL` and `Pandas`.

#### 1.1. `SQL JOINs` vs `pandas.DataFrame.join` or `pandas.DataFrame.merge`
Let's consider an example where we merge two public datasets available in BigQuery: the `bigquery-public-data.usa_names.usa_1910_2013` dataset, which contains information about names and their occurrences in the United States, and the `bigquery-public-data.usa_names.usa_1960_2017` dataset, which contains similar information but for a different time period. We will merge these datasets using the names as a common key.

In [ ]:
# Construct a reference to the "usa_names" dataset
dataset_ref = client.dataset("usa_names", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [ ]:
# List all the tables in the 'usa_names' dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there are two!)
for table in tables:  
    print(table.table_id)

We will look at the `usa_1910_2013` table.

In [ ]:
# Construct a reference to the "usa_1910_2013" table
table_ref = dataset_ref.table("usa_1910_2013")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

We will also look at the `usa_1910_current` table.

In [ ]:
# Construct a reference to the "usa_1910_current" table
table_ref = dataset_ref.table("usa_1910_current")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Now we will join both tables on `name` as the `key`.

#### 1.1.1. `SQL JOINs`

In [ ]:
# Define the query
join_query = """
WITH L AS
(
  SELECT name, SUM(number) AS total_number_1910_current
  FROM `bigquery-public-data.usa_names.usa_1910_current`
  GROUP BY name
  LIMIT 100
),

R AS 
(
  SELECT name, SUM(number) AS total_number_1910_2013
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE name IN (SELECT name FROM L)
  GROUP BY name
)

SELECT R.name, R.total_number_1910_2013, L.total_number_1910_current
FROM R
FULL JOIN L
ON L.name = R.name
ORDER BY L.total_number_1910_current DESC
"""

# Run the query, and return a pandas DataFrame
join_result = client.query(join_query).result().to_dataframe()
# Display number of rows and columns
print('(rows, cols):', join_result.shape)
# Preview of the top 5 rows
join_result.head()

### Code Explained!
The above code uses `CTEs (Common Table Expressions)` and `Aliases` in the `SQL` query. For more details, you can visit: 

https://www.kaggle.com/code/dansbecker/as-with

The above code simply pulls 100 common names available in each dataset and sums the total occurences of these names in each dataset. `Mary` is the most common name in the `USA` from `1910` to present. At least, the data says so!

#### 1.1.2. `pandas.DataFrame.join` or `pandas.DataFrame.merge`
We can achieve the same results using `Pandas`. The key difference between `Pandas` `join` and `merge` functions is that the `join` function joins datasets on a shared key column, if specified, which is named the same in each dataset and a column in the calling dataset and an index in the joining dataset, containing the shared key values. By default, it joins two or more datasets on the common indices if the key is not specified. The `merge` function allows SQL-style joining, where the key columns in each dataset can be specified separately. For more details, you can visit the following: 

[pandas.DataFrame.join](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html#pandas.DataFrame.join)

[pandas.DataFrame.merge](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html#pandas-dataframe-merge)


In [ ]:
# Define your first query to create dataframe for the first dataset
query_1 = """
SELECT name, SUM(number) AS total_number_1910_current
FROM `bigquery-public-data.usa_names.usa_1910_current`
GROUP BY name
LIMIT 100
"""

# Execute the first query and store the results in a Pandas DataFrame
query_job_1 = client.query(query_1)
df_1910_current = query_job_1.result().to_dataframe()

In [ ]:
# Define your second query to create dataframe for the second dataset
query_2 = """
SELECT name, SUM(number) AS total_number_1910_2013
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE name IN (SELECT name FROM `bigquery-public-data.usa_names.usa_1910_current` GROUP BY name LIMIT 100)
GROUP BY name
"""

# Execute the second query and store the results in another Pandas DataFrame
query_job_2 = client.query(query_2)
df_1910_2013 = query_job_2.result().to_dataframe()

__Note:__ We pulled only `100` rows from each dataset to conserve both memory and time since we are uncertain about the dataset sizes. If the datasets are significantly larger than the `CPU` usage and memory allocated for this notebook, the query might fail.

##### Using the `join` function:

In [ ]:
# Merge the two DataFrames on the 'name' column
df_join = df_1910_2013.join(df_1910_current.set_index('name'), 
                              on='name', 
                              how='inner').sort_values('total_number_1910_current', # sort by total_number_1910_current
                                                       ascending=False, # in descending
                                                       ignore_index=True) # ignore_index=True to reset index in the merged dataset

# Display number of rows and columns
print('(rows, cols):', df_join.shape)
# Display the top 5 rows of the merged DataFrame
df_join.head()

##### Using the `merge` function: 

In [ ]:
# Merge the two DataFrames on the 'name' column
df_merge = df_1910_2013.merge(df_1910_current,                                     # sort by total_number_1910_current
                              on='name').sort_values('total_number_1910_current', # by default how='inner' when using merge 
                                                       ascending=False, 
                                                       ignore_index=True) # ignore_index=True to reset index in the merged dataset

# Display number of rows and columns
print('(rows, cols):', df_merge.shape)
# Display the top 5 rows of the merged DataFrame
df_merge.head()

#### Code Explained!
We acheived the same results as `1.1.1.` but this time, we did not merge the datasets directly in the sql query, rather we pulled each datasets separately, then merged using both the `join` and `merge` functions in`Pandas`.

####  Let's Compare!
The dataframes we created so far should be all the same. Let's check, if this is true!

In [ ]:
# Check if the three dataframes are the same
join_result.equals(df_join) and df_join.equals(df_merge)

#### 1.2. `SQL UNIONs` vs `pandas.DataFrame.concat`
In the previous examples, we joined the datasets on a common `key`. This time, we will join them row-wise.

#### 1.2.1. `SQL` UNIONs

In [ ]:
# Define the query
union_query = """
 WITH L AS 
 (
  SELECT state, gender, name, year, number
  FROM `bigquery-public-data.usa_names.usa_1910_current`
  LIMIT 100
),

R AS 
(
  SELECT state, gender, name, year, number
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE name IN (SELECT name FROM L)
)

SELECT state, gender, name, year, number
FROM L
UNION ALL
SELECT state, gender, name, year, number
FROM R
ORDER BY state, gender, name, year, number
"""

# Run the query, and return a pandas DataFrame
union_result = client.query(union_query).result().to_dataframe()
# Display number of rows and columns
print('(rows, cols):', union_result.shape)
# Preview of the top 5 rows
union_result.head()

#### Code Explained!
The above code simply pulls `100` rows from the first dataset and the rows from the second dataset, where the names are common in the pulled `100` rows in the first dataset, and merges them into a single dataset.

#### 1.2.2. `pandas.DataFrame.concat`

By default, the `concat` function in Pandas merges rows of the datasets for common columns and appends any uncommon columns with their corresponding values to the corresponding rows. It fills in the missing values with `numpy.nan` at the corresponding index positions for columns with no values in the merged dataset. When the `axis` is set to `1`, it merges the columns of different datasets with values at their corresponding matching indices, leaving others as `numpy.nan`, similar to the join types described in section `1.1.`. If you set the `axis` to `1` and the `join` argument to `inner`, it will merge the row values of the columns with the same names in different datasets. Therefore, we can conclude that the `concat` function is the versatile data merging function in Pandas. For more infromation, you can visit the following:

[pandas.concat](https://pandas.pydata.org/docs/reference/api/pandas.concat.html#pandas-concat)

In [ ]:
# Define your first query to create dataframe for the first dataset
query_1 = """
SELECT state, gender, name, year, number
FROM `bigquery-public-data.usa_names.usa_1910_current`
LIMIT 100
"""

# Execute the first query and store the results in a Pandas DataFrame
query_job_1 = client.query(query_1)
df_1910_current = query_job_1.result().to_dataframe()

In [ ]:
# Define your second query to create dataframe for the second dataset
query_2 = """
SELECT state, gender, name, year, number
FROM `bigquery-public-data.usa_names.usa_1910_2013`
WHERE name IN (SELECT name FROM `bigquery-public-data.usa_names.usa_1910_current` LIMIT 100)
"""

# Execute the second query and store the results in another Pandas DataFrame
query_job_2 = client.query(query_2)
df_1910_2013 = query_job_2.result().to_dataframe()

In [ ]:
# Merge the rows of the two DataFrames
df_concat = pd.concat([df_1910_current, df_1910_2013]).sort_values(
                                                    ['state', 'gender', 'name', 'year', 'number'], ignore_index=True)
                                                              # sort by state, gender, name, year, number
                                                             # ignore_index=True to reset index in the merged dataset
# Display number of rows and columns
print('(rows, cols):', df_concat.shape)
# Display the top 5 rows of the merged DataFrame
df_concat.head()

#### Code Explained!
The above code simply pulls `100` rows from the first dataset and the rows from the second dataset, where the names are common in the pulled `100` rows in the first dataset into `Pandas` dataframes, and merge them into a single dataset.

####  Let's Compare!
The dataframes we created so far should be all the same. Let's check, if this is true!

In [ ]:
# Check if the two dataframes are the same
union_result.equals(df_concat)

#### 2. Advanced Analytics Functions
Here we will look at at least one example of different types of analytics functions can be defined in `SQL` and `Pandas`.

#### 2.1. Analytic aggregate functions
We will create a cumulative sum of each name by `year` in the datasets, `usa_1910_current` (`1000` data points).

#### 2.1.1. `SQL` analytic aggregate functions
We will define analytic aggregate funtions using `SQL`.

In [ ]:
# Define the query
query = """
WITH L AS
(
SELECT name, year, SUM(number) AS total_num
FROM `bigquery-public-data.usa_names.usa_1910_current`
GROUP BY name, year
LIMIT 1000
)

SELECT name, year, total_num,
    SUM(total_num) 
        OVER (
              PARTITION BY name
              ORDER BY year
              ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
             ) AS cum_sum
    FROM L
    ORDER BY name, year
"""

# Run the query, and return a pandas DataFrame
df_query = client.query(query).result().to_dataframe()
# Display number of rows and columns
print('(rows, cols):', df_query.shape)
# Preview of the top 5 rows
df_query.head()

#### Code Explained!
The above code simply pulls `1000` rows from the `usa_1910_current` dataset grouped by `name` and `year` and calculate a cummulative sum (cummulative total occurences) for each name from year to year ascending.

#### 2.1.2. Using `Pandas` for analytic aggregation
We will perform the above in a `Pandas` dataframe.

We will utilise the `pandas.DataFrame.groupby` function to accomplish this task. This function enables us to group data by one or more column values and apply various built-in aggregation functions provided by Pandas. Additionally, it allows us to apply custom functions using Pandas `lambda` expressions or user-defined functions. For more information, you can visit the following:

[pandas.DataFrame.groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html#pandas.DataFrame.groupby)

[pandas.core.groupby.DataFrameGroupBy.apply](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.apply.html#pandas.core.groupby.DataFrameGroupBy.apply)

In [ ]:
# Define your query to create dataframe
query = """
SELECT name, year, SUM(number) AS total_num
FROM `bigquery-public-data.usa_names.usa_1910_current`
GROUP BY name, year
LIMIT 1000
"""

# Execute the query and store the results in Pandas DataFrame
query_job = client.query(query)
df_1910_current = query_job.result().to_dataframe()

In [ ]:
# Calculate a cummulative sum for each name year to year
# Group by 'name', calculate cumulative sum, and reset index
df_grouped = (
    df_1910_current
    .groupby('name', as_index=False)
    .apply(lambda df: df.assign(cum_sum=df.total_num.cumsum()))
    .reset_index(drop=True)
)

# Display number of rows and columns
print('(rows, cols):', df_query.shape)
# Preview of the top 5 rows
df_grouped.head()

#### Code Explained!
The above code simply pulls `1000` rows from the `usa_1910_current` dataset grouped by `name` and `year` and calculate a cummulative sum (cummulative total occurences) for each name from year to year ascending using `Pandas` `groupby` and `DataFrameGroupBy.apply` function.

####  Let's Compare!
The dataframes we created so far should be all the same. Let's check, if this is true!

In [ ]:
# Check if the two dataframes are the same
df_query.equals(df_grouped)

#### 2.2. Analytic navigation functions
Here, we will find the least and most common names in the dataset, `usa_1910_current` (`1000` data points).

#### 2.2.1. `SQL` analytic navigation functions
We will define analytic navigation funtions using `SQL`.

In [ ]:
# Define the query
query = """
WITH L AS
(
SELECT name, year, SUM(number) AS total_num
FROM `bigquery-public-data.usa_names.usa_1910_current`
GROUP BY name, year
LIMIT 1000
)

SELECT DISTINCT year,
    FIRST_VALUE(name) 
        OVER (
              PARTITION BY year
              ORDER BY total_num
              ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
             ) AS least_common_name,
    LAST_VALUE(name) 
        OVER (
              PARTITION BY year
              ORDER BY total_num
              ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
             ) AS most_common_name
    FROM L
    ORDER BY year
"""

# Run the query, and return a pandas DataFrame
df_query = client.query(query).result().to_dataframe()
# Display number of rows and columns
print('(rows, cols):', df_query.shape)
print('The most common name(s) in this dataset:', df_query.most_common_name.unique())
# Preview of the top 5 rows
df_query.head()

#### Code Explained!
The above code simply pulls the least and most common names in the `usa` by `year` from`1000` rows from the `usa_1910_current` dataset grouped by `name` and `year`.
`Mary` is the most common name in the `usa` for the years present in this dataset!

#### 2.2.2. Using `Pandas` for analytic navigation
We will perform the above in a `Pandas` dataframe.

We will utilise the `pandas.DataFrame.groupby` function to accomplish this task. This function enables us to group data by one or more column values and apply various built-in aggregation functions provided by Pandas. Additionally, it allows us to apply custom functions using Pandas `lambda` expressions or user-defined functions. For more information, you can visit the following:

[pandas.DataFrame.groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html#pandas.DataFrame.groupby)

[pandas.core.groupby.DataFrameGroupBy.apply](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.apply.html#pandas.core.groupby.DataFrameGroupBy.apply)

In [ ]:
# Define your query to create dataframe
query = """
SELECT name, year, SUM(number) AS total_num
FROM `bigquery-public-data.usa_names.usa_1910_current`
GROUP BY name, year
LIMIT 1000
"""

# Execute the query and store the results in Pandas DataFrame
query_job = client.query(query)
df_1910_current = query_job.result().to_dataframe()

In [ ]:
# Find the least and most common names for each year
# Group by 'year', find the least and most common names, and reset index
df_grouped = df_1910_current.groupby('year', as_index=False).apply(
                lambda df: pd.Series({
                    'least_common_name': df.loc[df['total_num'].idxmin(), 'name'],
                    'most_common_name': df.loc[df['total_num'].idxmax(), 'name']
                })
            ).reset_index(drop=True)

# Display number of rows and columns
print('(rows, cols):', df_grouped.shape)
print('The most common name(s) in this dataset:', df_grouped.most_common_name.unique())
# Preview of the top 5 rows
df_grouped.head()

#### Code Explained!
The above code simply pulls the least and most common names in the `usa` by `year` from`1000` rows from the `usa_1910_current` dataset grouped by `name` and `year`using `Pandas`.
`Mary` is the most common name in the `usa` for the years present in this dataset!

####  Let's Compare!
The dataframes we created so far should be all the same. Let's check, if this is true!

In [ ]:
# Check if the two dataframes are the same
df_query.equals(df_grouped)

#### 2.3. Analytic numbering functions
Here, we will find the top `3` names by `year` in the dataset, `usa_1910_current` (`1000` data points).

#### 2.3.1. `SQL` analytic numbering functions
We will define analytic numbering funtions using `SQL`.

In [ ]:
# Define the query
query = """
WITH L AS
(
SELECT name, year, SUM(number) AS total_num
FROM `bigquery-public-data.usa_names.usa_1910_current`
GROUP BY name, year
LIMIT 1000
)

SELECT year,name
FROM
(
SELECT year,name,
    RANK() 
     OVER (
           PARTITION BY year 
           ORDER BY total_num DESC) AS name_rank
    FROM L) AS ranked_names

WHERE name_rank <= 3
ORDER BY year,name_rank
"""

# Run the query, and return a pandas DataFrame
df_query = client.query(query).result().to_dataframe()
# Display number of rows and columns
print('(rows, cols):', df_query.shape)
# Preview of the top 5 rows
df_query.head()

#### Code Explained!
The above code simply ranks the most common names in the `usa` by `year` from`1000` rows from the `usa_1910_current` dataset grouped by `name` and `year` and returns the top `3` names for each year.
The `RANK()` and `ROW_NUMBER()` have the same meanings, but the former one assigns the same rank for the same value of the ordering column, in this case `total_num`. For eaxample, if `2` lines have the same highest `total_num` they will be ranked `1`, and the next one in the partition will be ranked 2. But the `ROW_NUMBER()` functions rank in chronological order (1,2,3..etc.).

#### 2.3.2. Using `Pandas` for analytic numbering
We will perform the above in a `Pandas` dataframe.

We will utilise the `pandas.DataFrame.groupby` function to accomplish this task. This function enables us to group data by one or more column values and apply various built-in aggregation functions provided by Pandas. Additionally, it allows us to apply custom functions using Pandas `lambda` expressions or user-defined functions. For more information, you can visit the following:

[pandas.DataFrame.groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html#pandas.DataFrame.groupby)

[pandas.core.groupby.DataFrameGroupBy.apply](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.apply.html#pandas.core.groupby.DataFrameGroupBy.apply)

In [ ]:
# Define your query to create dataframe
query = """
SELECT name, year, SUM(number) AS total_num
FROM `bigquery-public-data.usa_names.usa_1910_current`
GROUP BY name, year
LIMIT 1000
"""

# Execute the query and store the results in Pandas DataFrame
query_job = client.query(query)
df_1910_current = query_job.result().to_dataframe()

In [ ]:
# Find the top 3 names for each year
# Group by 'name', find the top 3 names, and reset index
df_grouped = df_1910_current.groupby(['year']).apply(
                    lambda df: df.sort_values('total_num', ascending=False)[['year', 'name']].head(3)
).reset_index(drop=True)

# Display number of rows and columns
print('(rows, cols):', df_grouped.shape)
# Preview of the top 5 rows
df_grouped.head()

#### Code Explained!
The above code simply ranks the most common names in the `usa` by `year` from`1000` rows from the `usa_1910_current` dataset grouped by `name` and `year` and returns the top `3` names for each year using `Pandas`.

####  Let's Compare!
The dataframes we created so far should be all the same. Let's check, if this is true!

In [ ]:
# Check if the two dataframes are the same
df_query.equals(df_grouped)

#### 3. Nested and Repeated Data
Here, we will transoform complex datasets into something like regular datasets. The complex datasets contain `Nested` and `Repeated` data, which are `JSON` or `Python` dictonary like objects in a list in the rows of a table as explained in the `Background`[3].

For this part of this notebook, we will use `BigQurey's` `github_repos` dataset, containining `GitHub` data.

#### 3.1. Unnesting using `SQL`
We will define queries using `SQL` to unnest complex datasets.

In [ ]:
# Construct a reference to the "github_repos" dataset
dataset_ref = client.dataset("github_repos", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [ ]:
# List all the tables in the 'github_repos' dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there are nine!)
for table in tables:  
    print(table.table_id)

We will look at the `languages` table.

In [ ]:
# Construct a reference to the "languages" table
table_ref = dataset_ref.table("languages")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

In [ ]:
# Define the query
query = """
SELECT 
    lang.name as name, lang.bytes as bytes
FROM 
    `bigquery-public-data.github_repos.languages`,
UNNEST(language) AS lang
LIMIT 1000
"""

# Run the query, and return a pandas DataFrame
df_query = client.query(query).result().to_dataframe()
# Display number of rows and columns
print('(rows, cols):', df_query.shape)
# Preview of the top 5 rows
df_query.head()

#### Code Explained!
We simply unnested the objects from the `language` column and inserted them as separate rows(objects inside `[]`) and columns(objects inside `{}`) in the datasets.

#### 3.2. Using `Pandas` for analytic numbering
We will perform the above in a `Pandas` dataframe.

To achieve this we will define a function.

In [ ]:
# Define your query to create dataframe for the nested dataset
query = """
SELECT 
    language
FROM 
    `bigquery-public-data.github_repos.languages`
LIMIT 1000
"""

# Execute the query and store the results in Pandas DataFrame
query_job = client.query(query)
df_nested = query_job.result().to_dataframe()
# Display number of rows and columns
print('(rows, cols):', df_nested.shape)
# Preview of the top 5 rows
df_nested.head()

In [ ]:
# Define a function to apply the trasformation
def unnest_data(df_nested, column_to_unnest): # pass the dataframe and the column from which unnested
    
    #  Other columns with indices
    other_cols = df_nested.drop(columns=f'{column_to_unnest}')

    # Separated rows with the duplicate indices for objects {} that was in the same []
    nested_cols = df_nested.explode(column=f'{column_to_unnest}')
    
    # {} objects are transformed to dataframe
    normalised_cols = pd.json_normalize(nested_cols[f'{column_to_unnest}'])

    """
    The remaining columns with indices are joined along axis 1 (columns) with the normalized DataFrame. 
    In this process, additional rows may be generated. This is because, when the objects within {} are 
    extracted from [], the parent indices are duplicated. Consequently, the values from the other columns 
    will also be duplicated for these normalized rows or column values.
    """
    df_unnested = pd.concat([other_cols, normalised_cols], axis=1).reset_index(drop=True)
    
    return df_unnested

In [ ]:
# Apply the function and take 1000 rows
df_unnested = unnest_data(df_nested, 'language').iloc[:1000]
# Display number of rows and columns
print('(rows, cols):', df_unnested.shape)
# Preview of the top 5 rows
df_unnested.head()


Not as simple as the `SQL` query, right!

We can also directly apply the transformation to the dataframe.

In [ ]:
# Directly trasform the dataframe
#  Other columns with indices
other_cols = df_nested.drop(columns='language')

# Separated rows with the duplicate indices for objects {} that was in the same []
nested_cols = df_nested.explode(column=f'language')

# {} objects are transformed to dataframe
normalised_cols = pd.json_normalize(nested_cols['language'])

# Concatenate and return 1000 rows
df_unnested2 = pd.concat([other_cols, normalised_cols], axis=1).reset_index(drop=True).iloc[:1000]
# Display number of rows and columns
print('(rows, cols):', df_unnested2.shape)
# Preview of the top 5 rows
df_unnested2.head()

#### Code Explained!
We simply unnested the objects from the `language` column and inserted them as separate rows(objects inside `[]`) and columns(objects inside `{}`) in the datasets using `Pandas`.

####  Let's Compare!
The dataframes we created so far should be all the same. Let's check, if this is true!

In [ ]:
# Check if the two dataframes are the same
df_query.equals(df_unnested) and df_unnested.equals(df_unnested2)

### Conclusion
This notebook demonstrates the interchangeability of `SQL` and `Pandas` functionalities. The inspiration behind this notebook stems from my past experiences with data manipulation using `Pandas`, particularly when I needed to apply `SQL` analytics to non-SQL datasets such as spreadsheets or CSV files.

### References
[1]. Database.Guide(2016) _What is a Foreign Key?_ Available at: https://database.guide/what-is-a-foreign-key/, Accessed September 15, 2023.

[2]. Cook, Alexis. "Analytics Functions." Kaggle, Version 14.0, https://www.kaggle.com/code/alexisbcook/analytic-functions, Accessed September 15, 2023.

[3]. Cook, Alexis. "Nested and Repeated Data." Kaggle, Version 14.0, https://www.kaggle.com/code/alexisbcook/nested-and-repeated-data, Accessed September 15, 2023.

### Author
Khaled Ahmed

[LinkedIn](https://www.linkedin.com/in/ahmedkhaled40/)